# 9. Bayesian methods

The first step is to activate the packages and load the data.

In [1]:
import pyreadr 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime as dt
import sklearn


from plotnine import *
from plotnine.data import *
import seaborn as sns###설치
from scipy import stats
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import BayesianRidge

In [3]:
#data_ml = pyreadr.read_r(insert path of data_ml.RData)["data_ml"]
data_ml.date = pd.to_datetime(data_ml.date)

data_ml = data_ml.loc[(data_ml['date'] > "1999-12-31") & (data_ml['date'] < "2019-01-01")]
data_ml.sort_values(by = ['stock_id','date'], inplace = True)

We also copy/paste some chunks from Chapter 2.


In [4]:
stock_ids = data_ml.stock_id.unique() 
stock_days = data_ml.stock_id.value_counts().sort_index() 
stock_ids_short = stock_ids[stock_days.values == max(stock_days.values)]

returns_short = data_ml[data_ml['stock_id'].isin(stock_ids_short)][['date','stock_id','R1M_Usd']]
returns = returns_short.pivot_table(index=['date'],columns='stock_id',values='R1M_Usd')  
features = list(data_ml.columns[2:95]) 
features_short = ["Div_Yld", "Eps", "Mkt_Cap_12M_Usd", "Mom_11M_Usd", 
                    "Ocf", "Pb", "Vol1Y_Usd"]
data_ml['R1M_Usd_C'] = data_ml.R1M_Usd >  data_ml.groupby('date').R1M_Usd.transform('median')
data_ml['R12M_Usd_C'] = data_ml.R12M_Usd >  data_ml.groupby('date').R12M_Usd.transform('median')
separation_date = dt.strptime("2014-01-15","%Y-%m-%d")
training_sample = data_ml[data_ml.date < separation_date]
testing_sample = data_ml[data_ml.date  >= separation_date]

First, We move towards naive Bayes classifiers.

In [15]:
gauss_features_train = training_sample[features_short]
gauss_features_train *= 0.999 # Features smaller than 1
gauss_features_train += 0.0001 # Features larger than 0
gauss_features_train = stats.norm.ppf(gauss_features_train)  # Inverse Gaussian cdf

fit_NB_gauss = GaussianNB()
fit_NB_gauss.fit(gauss_features_train, training_sample.R1M_Usd_C)

GaussianNB(priors=None, var_smoothing=1e-09)

In [16]:
gauss_features_test = testing_sample[features_short]
gauss_features_test *= 0.999 
gauss_features_test += 0.0001 
gauss_features_test = stats.norm.ppf(gauss_features_test)  

print("Hit ratio:", (fit_NB_gauss.predict(gauss_features_test )  == testing_sample.R1M_Usd_C ).mean())

Hit ratio: 0.4956984958979034


Finally, we end with Bayesian additive trees
To do this, you should download barpy library detained in 

https://github.com/JakeColtman/bartpy

In [17]:
from bartpy.sklearnmodel import SklearnModel

Once the model is trained, we evaluated its performance. We simply compute the hit ratio. 


In [18]:
fit_bart = SklearnModel(
                        n_trees=20,       # Number of trees in the model 
                        n_burn=100,       # Size of burn-in sample
                        n_samples=200,    # Number of posteriors drawn
                        beta=2,           # beta in the tree structure prior
                        alpha=0.95 )      # alpha in the tree structure prior
fit_bart.fit(training_sample[features_short][:100000], training_sample.R1M_Usd[:100000]) #shorten the training sample to avoid shutting down

SklearnModel(alpha=0.95, beta=2, initializer=None, n_burn=100, n_chains=4,
             n_jobs=-1, n_samples=200, n_trees=20, sigma_a=0.001, sigma_b=0.001,
             store_acceptance_trace=False, store_in_sample_predictions=False,
             thin=0.1,
             tree_sampler=<bartpy.samplers.unconstrainedtree.treemutation.UnconstrainedTreeMutationSampler object at 0x000002572761AD68>)

Once the model is trained,^[In the case of BARTs, the training is consists exactly in the drawing of posterior samples.] we evaluated its performance. We simply compute the hit ratio. The predictions are embedded within the fit variable, under the name '*yhat.test*'.

In [19]:
print("Hit ratio:", (fit_bart.predict(testing_sample[features_short])* testing_sample.R1M_Usd > 0).mean())

Hit ratio: 0.5460346399270738
